# PPP Loan Analysis
Team 22032: Danny Rivas, Javan Reuto

Date: 03/05/22

---

In [8]:
import pandas as pd
import numpy as np
from numpy import nan
import re
import datetime as dt

Before we begin, here is an overview what we'll cover:
* [**Part One: Data Preparation**](#part01)
    * [Load the Data](#load)
* [**Part Two: Analysis**](#part02)
* [**Part Three: Prediction**](#part03)

## Part One: Data Preparation <a class="anchor" id="part01"></a>

### Load the Data <a class="anchor" id="load"></a>

In [12]:
# Load csv file as dataframe
ga_full_df = pd.read_csv('ppp_applicants_ga_full.csv')
ga_full_df.head(5)

,name,amount,state,address,city,zip,naics_code,business_type,jobs_retained,date_approved,...,project_county_name,project_state,project_zip,originating_lender_city,originating_lender_state,loan_status_date,originating_lender_location_id,lmi_indicator,forgiveness_amount,forgiveness_date
0,LUMMUS CORPORATION,2000000.0,GA,225 Bourne Blvd,Savannah,31408-9586,333249.0,Corporation,294,2021-02-20,...,CHATHAM,GA,31408-9586,CHICAGO,IL,NaN,116998,N,NaN,NaN
1,COLIANT SOLUTIONS INC.,1294555.0,GA,2703 Brickton North Dr,Buford,30518-9101,541519.0,Corporation,63,2021-01-31,...,GWINNETT,GA,30518-9101,COLUMBUS,GA,NaN,19248,Y,NaN,NaN
2,YOHE PLUMBING INC,729509.0,GA,1120 Franke Court N/A,Augusta,30909,238220.0,Corporation,105,2021-03-12,...,RICHMOND,GA,30909,PHOENIXVILLE,PA,NaN,121536,N,NaN,NaN
3,LEWIS COLOR LITHOGRAPHERS INC,571193.4,GA,30 Joe Kennedy Blvd,Statesboro,30458-3417,323111.0,Corporation,49,2021-02-02,...,BULLOCH,GA,30458-3417,COLUMBUS,GA,NaN,19248,N,NaN,NaN
4,"ALMA PAK INTERNATIONAL, LLC",472700.0,GA,230 PINEVIEW RD,ALMA,31510-4326,445230.0,Partnership,37,2020-04-10,...,BACON,GA,31510-4326,ALMA,GA,2021-01-08,17811,N,475877.59,2020-12-15


In [13]:
ga_full_df.nunique()

name                              444162
amount                            106476
state                                  1
address                           429791
city                                2762
zip                               229592
naics_code                          1086
business_type                         23
jobs_retained                        462
date_approved                        237
lender                              1194
congressional_district                38
loan_number                       553828
sba_office_code                       19
processing_method                      2
loan_status                            3
term                                  21
sba_guaranty_percentage                1
initial_approval_amount           105931
current_approval_amount           106476
undisbursed_amount                   110
servicing_lender_location_id        1243
servicing_lender_name               1184
servicing_lender_address            1232
servicing_lender

In [10]:

ga_full_df = ga_full_df.set_index('loan_number')


name                               object
amount                            float64
state                              object
address                            object
city                               object
zip                                object
naics_code                        float64
business_type                      object
jobs_retained                       int64
date_approved                      object
lender                             object
congressional_district             object
sba_office_code                     int64
processing_method                  object
loan_status                        object
term                                int64
sba_guaranty_percentage             int64
initial_approval_amount           float64
current_approval_amount           float64
undisbursed_amount                float64
servicing_lender_location_id        int64
servicing_lender_name              object
servicing_lender_address           object
servicing_lender_city             

In [7]:
# Creating function to convert dates to datetime format
def format_date(df, column):
    df[column] = pd.to_datetime(df[column], format="%Y-%m-%d")

# Running function on date columns
format_date(ga_full_df,'date_approved')
format_date(ga_full_df,'loan_status_date')
format_date(ga_full_df,'forgiveness_date')

#
ga_full_df['loan_number'] = ga_full_df['loan_number'].astype(object)
ga_full_df = ga_full_df.set_index('loan_number')

0        2021-02-20
1        2021-01-31
2        2021-03-12
3        2021-02-02
4        2020-04-10
            ...    
553823   2021-04-30
553824   2021-04-02
553825   2021-02-19
553826   2020-08-03
553827   2021-02-04
Name: date_approved, Length: 553828, dtype: datetime64[ns]

## Part Two: Analysis <a class="anchor" id="part02"></a>

## Part Three: Prediction <a class="anchor" id="part03"></a>